#### MediLake Health  EHR MERGE, SCD & Business Logic

MediLake Health – EHR MERGE, SCD & Business Logic


##### 1.1 Create Empty SCD Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS medilakehealth_catalog.silver.patients_scd (
  patient_id STRING,
  first_name STRING,
  last_name STRING,
  gender STRING,
  date_of_birth DATE,
  city STRING,
  start_date DATE,
  end_date DATE,
  is_current BOOLEAN
)
USING DELTA;


##### 1.2 MERGE Logic (SCD Type-2)

In [0]:
%sql

MERGE INTO medilakehealth_catalog.silver.patients_scd tgt
USING (
  SELECT
    patient_id,
    first_name,
    last_name,
    gender,
    date_of_birth,
    city,
    CAST(updated_at AS DATE) AS effective_date
  FROM medilakehealth_catalog.bronz.patients
) src
ON tgt.patient_id = src.patient_id
AND tgt.is_current = true

WHEN MATCHED AND (
  tgt.city <> src.city OR
  tgt.first_name <> src.first_name OR
  tgt.last_name <> src.last_name
)
THEN UPDATE SET
  tgt.end_date = src.effective_date - INTERVAL 1 DAY,
  tgt.is_current = false

WHEN NOT MATCHED
THEN INSERT (
  patient_id, first_name, last_name, gender,
  date_of_birth, city,
  start_date, end_date, is_current
)
VALUES (
  src.patient_id, src.first_name, src.last_name, src.gender,
  src.date_of_birth, src.city,
  src.effective_date, NULL, true
);


##### Clean Encounters – encounters_clean
Business Rules :
- Remove duplicates
- Keep latest record per encounter
- Prepare for analytics
- 

##### 2.1 Deduplicate Using Window Function

In [0]:
%sql 

CREATE OR REPLACE TABLE medilakehealth_catalog.silver.encounters_clean
USING DELTA
AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY encounter_id
           ORDER BY updated_at DESC
         ) AS rn
  FROM medilakehealth_catalog.bronz.encounters
)
WHERE rn = 1;

 /* Why not MERGE here?
Because first load = full refresh → CREATE OR REPLACE */

#####  Diagnoses Clean

In [0]:
%sql 
CREATE OR REPLACE TABLE medilakehealth_catalog.silver.diagnoses_clean
USING DELTA
AS
SELECT
  diagnosis_id,
  encounter_id,
  patient_id,
  diagnosis_code,
  diagnosis_desc,
  diagnosis_date
FROM medilakehealth_catalog.bronz.diagnoses
WHERE diagnosis_code IS NOT NULL;


###### Lab Results – Append + Dedup


In [0]:
%sql
CREATE OR REPLACE TABLE medilakehealth_catalog.silver.lab_results_clean
USING DELTA
AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY lab_result_id
           ORDER BY result_date DESC
         ) AS rn
  FROM medilakehealth_catalog.bronz.lab_results
)
WHERE rn = 1;


##### Medications – Active Medication Logic

In [0]:
%sql
CREATE OR REPLACE TABLE medilakehealth_catalog.silver.medications_clean
USING DELTA
AS
SELECT
  medication_id,
  patient_id,
  medication_name,
  dosage,
  start_date,
  end_date,
  CASE
    WHEN end_date IS NULL THEN true
    ELSE false
  END AS is_active
FROM medilakehealth_catalog.bronz.medications;


#### Validate Silver Tables

In [0]:
%sql
SELECT * FROM medilakehealth_catalog.silver.patients_scd WHERE is_current = true;




In [0]:
%sql 
SELECT COUNT(*) FROM medilakehealth_catalog.silver.encounters_clean;


In [0]:
%sql
SELECT * FROM medilakehealth_catalog.silver.medications_clean WHERE is_active = true;